## Wczytywanie potrzebnych bibliotek.

In [40]:
import pandas as pd
import pathlib as path
import numpy as np


### Odczytywanie popularnych imion i nazwisk żeńskich i męskich.

In [41]:
# Znajdź katalog z imionami i nazwiskami.
name_dir_path: path.Path = path.Path().cwd()/"Imiona i nazwiska"

# Znajdź  katalog, gdzie zapisujemy wszystkie tabele .csv.
csv_tables_dir_path: path.Path = path.Path().cwd()/"Tabele_csv"


def read_names(first_names_file:str, last_names_file:str, dir_path:path.Path) -> tuple[pd.Series, pd.Series]:
    """Funkcja odczytuje pliki zawierające najpopularniejsze polskie imiona i nazwiska. Wczytuje je do szeregu danych typu pandas"""
    # Odczytaj imiona oraz nazwiska.
    first_names: pd.DataFrame = pd.read_excel(dir_path/first_names_file  ,"Arkusz1",
                                                usecols = [0,2]
                                                )
    last_names: pd.DataFrame = pd.read_excel(dir_path/last_names_file, "Arkusz1",
                                                nrows = 5_000) # Wierszy w arkuszu jest bardzo dużo (ponad 380 000). 
                                                                # Odczytaj tylko pewną część wierszy.
                                                                

    # Bierzemy pod uwagę jedynie najpopularniejsze imiona i nazwiska.
    first_names = first_names.loc[first_names["LICZBA_WYSTĄPIEŃ"] > 500, "IMIĘ_PIERWSZE"]
    last_names = last_names.loc[last_names["Liczba"] > 500, "Nazwisko aktualne"] 

    # Skróc nazwy kolumn.
    first_names.columns = ['imię']
    last_names.columns = ['nazwisko']

    return first_names, last_names

# Wczytaj męskie imiona i nazwiska.
male_first_names, male_last_names = read_names("imiona_meskie.xlsx", "nazwiska_meskie.xlsx", name_dir_path)

# Wczytaj żęńskie imiona i nazwiska.
female_first_names, female_last_names = read_names("imiona_zenskie.xlsx", "nazwiska_zenskie.xlsx", name_dir_path)


### Tworzenie danych osobowych pracowników.

In [ ]:
def generate_worker_personal_data(first_names:pd.Series, last_names:pd.Series, trades_n:dict[str,int], email_domain:str) -> pd.DataFrame:
    """Funkcja generuje dane osobiste pracowników. Tworzone są pary imię-nazwisko, numer telefonu oraz adres e-mail firmowy.

    """
    # Przygotuj wyjściową ramkę danych.
    workers_df: pd.DataFrame = pd.DataFrame(data = {"imię":pd.Series(dtype = pd.StringDtype()),
                                                    "nazwisko": pd.Series(dtype = pd.StringDtype()), 
                                                    "stanowisko":pd.Series(dtype = pd.CategoricalDtype(categories = ["kierowca","przewodnik"],
                                                                                                  ordered = False
                                                                                                  )),
                                                    "numer telefonu":pd.Series(dtype = pd.StringDtype()), 
                                                    "adres email": pd.Series(dtype = pd.StringDtype())},
                                                
    )

    # Lista wszystkich cyfr. Przyda się przy generowaniu numerów telefonów.
    digits:list[str] = ["0","1","2","3","4","5","6","7","8","9"]



    id: int = 0
    for pos in trades_n:
        for _ in range(trades_n[pos]):
            rand_first_name_idx:int = np.random.randint(0, first_names.shape[0])
            rand_last_name_idx:int = np.random.randint(0, last_names.shape[0])

            rand_first_name:str = first_names[rand_first_name_idx].capitalize() # Wylosuj imię
            rand_last_name:str = last_names[rand_last_name_idx].capitalize()  # Wylosuj nazwisko.

            # Wygeneruj losowy numer telefonu nie zaczynający się od 0.
            phone_number:np.array = np.random.choice(digits, 9)

            # Jeżeli wylosowaliśmy zero, podmień cyfrę.
            if phone_number[0] == "0":
                phone_number[0] = np.random.choice(digits[1:], 1)[0]

            phone_number: str = "".join(phone_number)

            # Stwórz email dla tego pracownika
            email:str = rand_first_name.lower() + rand_last_name.lower() + "".join(np.random.choice(digits, 3)) +"@" + email_domain
            

            workers_df.loc[id, :] = [rand_first_name, rand_last_name, pos ,phone_number, email]

            id += 1

    return workers_df

def insert_id_col(people_data:pd.DataFrame, col_name:str) -> pd.DataFrame:
    id_column:pd.Series = pd.Series(data = range(1, len(people_data)+1),
                                    name = col_name, dtype = pd.Int16Dtype())
    people_data.insert(loc = 0,column = id_column.name,
                       value = id_column)
    
    return people_data


# Dwa razy więcej przewodników niż mężczyzn.
female_drivers:int = 15
females_guides: int = 2*female_drivers

male_drivers:int = 25
male_guides = 2*male_drivers

female_trades_n: dict[str, int] = {"przewodnik":females_guides, 
                                   "kierowca":female_drivers}

male_trades_n: dict[str, int] = {"przewodnik":male_guides, 
                                 "kierowca":male_guides}


# Wygeneruj dane osobiste dla pracowniczek
email_domain = "nowowiejska.com"
female_workers_data:pd.DataFrame = generate_worker_personal_data(female_first_names, female_last_names, female_trades_n, email_domain)

male_workers_data: pd.DataFrame = generate_worker_personal_data(male_first_names, male_last_names, male_trades_n, email_domain)

# Połącz kobiecych i męskich pracowników.
workers_data:pd.DataFrame = pd.concat(objs = [female_workers_data, male_workers_data], axis = 0).reset_index(drop = True)
workers_data:pd.DataFrame = insert_id_col(workers_data, "id_pracownika")

# Zapisz dane pracowników do pliku .csv.
workers_data.to_csv(path_or_buf = csv_tables_dir_path/"Pracownicy.csv",
                    sep = ";", index = False)
